This was an initial attempt at converting sound to text with free packages, but it wasn't as good as the paid Google Cloud Speech-to-Text API, so the idea ended up being discarded

In [1]:
# Using google's speech recognition
import speech_recognition as sr

In [2]:
audio_filename = "Observador_podcasts\salazar-criou-o-servico-nacional-de-saude.wav"
r = sr.Recognizer()
with sr.AudioFile(audio_filename) as source:
    audio = r.record(source)

In [3]:
text = r.recognize_google(audio, language='pt-PT')  # Replace 'pt-PT' with the appropriate language code
print(text)

Sim está a ouvir agora a rádio observador está no ar O que é que sei quê das manhãs 360 com o jornalista Teresa Borges 3 a Bom dia da festa hoje falamos do serviço nacional de saúde falamos do SMS para verificar uma publicação que anda a circular nas redes sociais e que afirma que foi António de Oliveira Salazar que criou o serviço nacional de saúde durante o período do estado novo nesta publicação o autor questiona Porque razão dizem que o SNS foi montado pela democracia há 40 anos quando acordo com este post foi exatamente o Estado Novo Salazar que o implemento para justificar esta afirmação o autor da publicação recorda a obra feita por Salazar no centro de saúde enumera algumas unidades hospitalares que foram construídas nesta fase bem como medidas implementadas é o caso da instituição do abono de família para todos os filhos de pais a falar mas trazem conhecimento público os que os registos os de uma outra data para a criação do SNS e ela é posteriores ao estado novo tratamento de

In [27]:
# Using openAI's whisper
import whisper

model = whisper.load_model("base")
result = model.transcribe("salazar-criou-o-servico-nacional-de-saude.wav")
print(result["text"])

FileNotFoundError: [WinError 2] The system cannot find the file specified

: 

In [20]:
# This is definitely a good start, but it would be better if we could perform speech diarization
# Speech diarization allows us to separate what each speaker is saying into different paragraphs,
# which should help identify the original statement identified as false that is mentioned in the podcast
from pyAudioAnalysis import audioSegmentation
import numpy as np

audio_file = 'Observador_podcasts\salazar-criou-o-servico-nacional-de-saude.wav'

# Perform speaker diarization
result = audioSegmentation.speaker_diarization(audio_file, n_speakers=3)

print(result)

# Retrieve segments and corresponding speakers
segments = result[0]  # Access the first element of the tuple
speakers = np.unique(segments)  # Get unique speaker labels

# Print segments and corresponding speakers
for segment, speaker in zip(segments, speakers):
    print(f"Segment: {segment}  Speaker: {speaker+1}")

c:\Users\UserPC\anaconda3\envs\tf-gpu-share-backup\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\UserPC\anaconda3\envs\tf-gpu-share-backup\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(array([1, 1, 2, ..., 1, 1, 1], dtype=int64), -1, -1)
Segment: 1  Speaker: 1
Segment: 1  Speaker: 2
Segment: 2  Speaker: 3


In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

# Give the file path to your audio file
audio_file_path = 'Observador_podcasts\salazar-criou-o-servico-nacional-de-saude.wav'
wav_fpath = Path(audio_file_path)

# preprocess_wav uses a VAD to trim out the silences in the audio file and also normalizes the decibel level of audio.
wav = preprocess_wav(wav_fpath)

# Create an instance of the VoiceEncoder class
encoder = VoiceEncoder("cpu")

# cont_embeds is a N by D matrix, where N is the number of segments or d-vectors created and D is the dimension of each d-vector
# wav_splits is a list with the start and end time of each window for which a d-vector has been created
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

In [7]:
from spectralcluster import SpectralClusterer

# Set d-vectors' labels
clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100)
    #p_percentile=0.90,
    #gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)

c:\Users\UserPC\anaconda3\envs\tf-gpu-share-backup\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [21]:
import pyAudioAnalysis